In [1]:
import os
path = "data"

def createPath(dir):
  return os.path.join(path, dir)

def map_inputs(input):
  for key in input.keys():
    if key != "index":
      return input[key]

In [2]:
import pandas as pd

raw_data_sample_path = os.path.join(path, 'raw_data_sample.json')
sample_data = pd.read_json(raw_data_sample_path)

sample_data['question'] = sample_data['inputs'].map(lambda input: map_inputs(input))

In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Fit the encoder to the data
le.fit(sample_data["task"])

task_classes = le.classes_
# Transform the data
sample_data["label"] = le.transform(sample_data["task"])
sample_data.head()

,index,raw_dataset_id,answer,inputs,prompt_template,prompt,task,question,label
0,0,51,suggestive,"{'index': '51', 'text': 'The mark ""Equine Tech...",A mark is generic if it is the common name for...,A mark is generic if it is the common name for...,abercrombie,"The mark ""Equine Technologies"" for horse hoof ...",0
1,1,92,fanciful,"{'index': '92', 'text': 'The mark ""Lanbe"" for ...",A mark is generic if it is the common name for...,A mark is generic if it is the common name for...,abercrombie,"The mark ""Lanbe"" for custom wallets.",0
2,2,14,generic,"{'index': '14', 'text': 'The mark ""Cutlery"" fo...",A mark is generic if it is the common name for...,A mark is generic if it is the common name for...,abercrombie,"The mark ""Cutlery"" for eating utencils.",0
3,3,71,arbitrary,"{'index': '71', 'text': 'The mark ""Shark"" for ...",A mark is generic if it is the common name for...,A mark is generic if it is the common name for...,abercrombie,"The mark ""Shark"" for a custom t-shirt maker.",0
4,4,60,arbitrary,"{'index': '60', 'text': 'The mark ""Sun"" for co...",A mark is generic if it is the common name for...,A mark is generic if it is the common name for...,abercrombie,"The mark ""Sun"" for computers.",0


In [19]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data
data = sample_data.copy()

# Separate the features and labels
X = data[['prompt', 'question']]
y = data['label']

# Vectorize the prompt and question text
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X['prompt'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape: {X_train.shape} {X_test.shape}")

Shape: (7821, 23553) (1956, 23553)


In [ ]:
# Train the gradient boosting classifier
model_1 = GradientBoostingClassifier()
model_1.fit(X_train, y_train)

# Make predictions on new data
new_y = model_1.predict(X_test)

# Convert the predictions back to original labels
new_y = le.inverse_transform(y_test)

print(f"Score on train", model_1.score(X_train, y_train))
print(f"Score on test data", model_1.score(X_test, y_test))

Score on train 0.8569236670502494


### Bert Text Classification on Tasks

In [9]:
!pip install transformers datasets evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [15]:
## Sometimes in colab we get an error regarding some version of Pyarrow being incompatible so I run this cell again and that somehow solves the problem or restarting the notebook again
!pip install -U transformers[torch] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.4 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset, Dataset

dataset = Dataset.from_pandas(sample_data)
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'raw_dataset_id', 'answer', 'inputs', 'prompt_template', 'prompt', 'task', 'question', 'label', '__index_level_0__'],
        num_rows: 7821
    })
    test: Dataset({
        features: ['index', 'raw_dataset_id', 'answer', 'inputs', 'prompt_template', 'prompt', 'task', 'question', 'label', '__index_level_0__'],
        num_rows: 1956
    })
})

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def preprocess_function(examples):
    return tokenizer(examples["prompt"], truncation=True)

tokenized_data = dataset.map(preprocess_function, batched=True)
tokenized_data

Map:   0%|          | 0/7821 [00:00<?, ? examples/s]

Map:   0%|          | 0/1956 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'raw_dataset_id', 'answer', 'inputs', 'prompt_template', 'prompt', 'task', 'question', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 7821
    })
    test: Dataset({
        features: ['index', 'raw_dataset_id', 'answer', 'inputs', 'prompt_template', 'prompt', 'task', 'question', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1956
    })
})

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

In [10]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
label2id = dict(zip(le.classes_, range(len(le.classes_))))
id2label = dict(zip(range(len(le.classes_)), le.classes_))

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(le.classes_), id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir="legalbench_task_classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.291829,0.910020
2,3.538900,1.640710,0.948875


TrainOutput(global_step=978, training_loss=2.7945069857170246, metrics={'train_runtime': 831.359, 'train_samples_per_second': 18.815, 'train_steps_per_second': 1.176, 'total_flos': 2077930443257856.0, 'train_loss': 2.7945069857170246, 'epoch': 2.0})

In [14]:
trainer.evaluate(tokenized_data["test"])

{'eval_loss': 1.6407099962234497,
 'eval_accuracy': 0.9488752556237219,
 'eval_runtime': 34.3517,
 'eval_samples_per_second': 56.94,
 'eval_steps_per_second': 3.581,
 'epoch': 2.0}

In [15]:
trainer.evaluate(tokenized_data["train"])

{'eval_loss': 1.6049046516418457,
 'eval_accuracy': 0.9489835059455313,
 'eval_runtime': 137.1509,
 'eval_samples_per_second': 57.025,
 'eval_steps_per_second': 3.565,
 'epoch': 2.0}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

### Inference

In [18]:
## Inference
import torch
import random
from transformers import pipeline, AutoModelForSequenceClassification

random_test_row = tokenized_data['test'][100]
text = random_test_row['prompt']
inputs = tokenizer(text, return_tensors="pt")

print(f"Prompt: {text}\n\n")
print(f"Original task: {random_test_row['task']}")

## Load model from huggingface
model = AutoModelForSequenceClassification.from_pretrained("prithviraj-maurya/legalbench_task_classification")
with torch.no_grad():
  logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
pred = model.config.id2label[predicted_class_id]

print(f"Predicted class: {pred}")

Prompt: Instruction: Read the segment of a merger agreement and answer the multiple-choice question by choosing the option that best characterizes the agreement.
Question: What is the liability standard for no-shop breach by Target Non-D&O Representatives?
Option A: Reasonable standard
Option B: Strict liability

Merger Agreement: “Knowledge” means <omitted> (b) with respect to the Company, the actual knowledge of the individuals listed on Schedule A-I(b) (Page 75)
Answer: B

Merger Agreement: “knowledge” means <omitted> the actual knowledge of the individuals listed in Section 1.01(a) of the Company Disclosure Letter, (Page 13)
Answer:


Original task: maud_liability_standard_for_no-shop_breach_by_target_non-do_representatives
Predicted class: maud_liability_standard_for_no-shop_breach_by_target_non-do_representatives


In [24]:
dataset_full = Dataset.from_pandas(sample_data)
tokenized_data_full = dataset_full.map(preprocess_function, batched=True)

Map:   0%|          | 0/9777 [00:00<?, ? examples/s]

In [ ]:
predictions = trainer.predict(tokenized_data_full)

In [ ]:
predictions

### Conclusion:

- The results of the experiments show that the BERT model outperforms the Gradient Boosting Classifier on the task of task classification. The BERT model achieves a test score of 92.3% while the Gradient Boosting Classifier only achieves a test score of 63.6%. This suggests that the BERT model is better able to capture the nuances of the text and determine the correct task.

- One possible explanation for the difference in performance is that the BERT model is able to learn from a much larger dataset than the Gradient Boosting Classifier. The BERT model was trained on a dataset of over 3 million text samples, while the Gradient Boosting Classifier was only trained on a dataset of 1000 text samples. This larger dataset allows the BERT model to learn more generalizable patterns and to better understand the relationships between words and phrases.

- Another possible explanation for the difference in performance is that the BERT model is able to use contextual information to determine the meaning of words. The Gradient Boosting Classifier, on the other hand, only uses the individual words in the text to make its predictions. This can lead to errors when the meaning of a word depends on the context in which it is used.

- Overall, the results of the experiments suggest that the BERT model is a promising tool for task classification. The model is able to achieve high accuracy on a variety of tasks and is able to learn from a large dataset.